# **Feature Engineering**

## Objectives

* To feature engineer for future ML tasks

## Inputs

* The data file, "US_Accidents_For_Feature_Eng.csv", which is locally saved in "Data/Feature_Eng"

## Outputs

* The csv file, "US_Accidents_For_ML.csv", which is locally saved in "Data/ML"

## Summary of Steps

* If you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\sonia\\Documents\\VS Studio Projects\\US_Accidents_ML_Project\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\sonia\\Documents\\VS Studio Projects\\US_Accidents_ML_Project'

---

## Required Libraries

In [47]:
import pandas as pd
import numpy as np
import re

---

## Load the Dataset

I load the dataset using Pandas.

In [22]:
df = pd.read_csv("Data/Feature_Eng/US_Accidents_For_Feature_Eng.csv")
pd.set_option("display.max_columns", None)
df

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),City,County,State,Timezone,Airport_Code,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Clearance_Time(hr),Clearance_Class
0,2,2022-09-08 20:54:00,2022-09-09 23:06:21,32.456486,-93.774536,0.501,Shreveport,Caddo,LA,US/Central,KSHV,78.0,78.0,62.0,29.61,10.00,CALM,0.0,0.00,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,26.205833,Very Long
1,2,2021-05-22 00:40:30,2021-05-25 09:56:58,36.804693,-76.189728,0.253,Virginia Beach,Virginia Beach,VA,US/Eastern,KORF,54.0,54.0,90.0,30.40,7.00,CALM,0.0,0.00,Fair,False,False,True,False,False,False,False,False,False,False,False,True,False,Night,81.274444,Very Long
2,2,2022-01-21 14:25:00,2023-01-21 16:10:00,29.895741,-90.090026,1.154,Marrero,Jefferson,LA,US/Pacific,KAUD,40.0,33.0,58.0,30.28,10.00,N,10.0,0.00,Mostly Cloudy,False,False,False,False,True,False,False,False,False,False,False,False,False,Day,8761.750000,Very Long
3,2,2020-11-27 00:44:00,2020-11-28 04:49:48,32.456459,-93.779709,0.016,Shreveport,Caddo,LA,US/Central,KSHV,62.0,62.0,75.0,29.80,10.00,SSE,8.0,0.00,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,28.096667,Very Long
4,2,2020-09-21 12:07:00,2020-09-22 15:22:36,26.966433,-82.255414,0.057,Port Charlotte,Charlotte,FL,US/Eastern,KPGD,84.0,84.0,69.0,29.99,10.00,E,18.0,0.00,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,27.260000,Very Long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,2021-06-10 00:18:00,2021-06-10 10:53:16,39.573795,-86.618947,4.314,Stilesville,Morgan,IN,US/Eastern,KIND,72.0,72.0,91.0,29.14,10.00,SE,5.0,0.00,Mostly Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night,10.587778,Long
9996,2,2020-12-16 14:49:30,2020-12-16 22:48:00,40.001124,-75.342886,0.634,Bryn Mawr,Delaware,PA,US/Eastern,KLOM,26.0,15.0,92.0,29.81,0.75,ENE,13.0,0.00,Snow,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,7.975000,Long
9997,2,2022-12-26 14:44:37,2022-12-27 00:16:30,34.988932,-85.493085,1.560,Guild,Marion,TN,US/Eastern,KCHA,33.0,33.0,35.0,29.51,10.00,SSW,3.0,0.00,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,9.531389,Long
9998,2,2020-11-14 18:44:30,2020-11-15 03:54:02,40.773530,-74.034951,0.686,Union City,Hudson,NJ,US/Eastern,KNYC,49.0,49.0,41.0,30.13,10.00,CALM,0.0,0.00,Fair,False,False,True,False,False,False,False,False,True,True,False,True,False,Night,9.158889,Long


---

## Detailed Look at Categorical Variables

In this section, I'm going to have a detailed look at the categorical variables and decide which ones to modify, drop or carry forward. For numerical variables, I will similarly decide which variables to modify, drop or carry forward after EDA.

In [23]:
for col in df.select_dtypes(include="object").columns:
    print(f"{col}: {df[col].nunique()}")

Start_Time: 9791
End_Time: 9920
City: 2433
County: 754
State: 48
Timezone: 4
Airport_Code: 993
Wind_Direction: 23
Weather_Condition: 51
Sunrise_Sunset: 2
Clearance_Class: 4


First, I would like to strip "US/" from each value under "Timezone".

In [24]:
df["Timezone"].unique()

array(['US/Central', 'US/Eastern', 'US/Pacific', 'US/Mountain'],
      dtype=object)

In [25]:
df["Timezone"] = df["Timezone"].str.replace("US/", "", regex=False)
df["Timezone"].value_counts()

Timezone
Eastern     4298
Pacific     2521
Central     2447
Mountain     734
Name: count, dtype: int64

Next, I will take a look at the values in "Sunrise_Sunset" and ensure they are valid.

In [26]:
df["Sunrise_Sunset"]. value_counts()

Sunrise_Sunset
Day      6445
Night    3555
Name: count, dtype: int64

Similarly, I will take a look at values for "Wind_Direction".

In [27]:
df["Wind_Direction"].value_counts()

Wind_Direction
CALM        1770
S            799
N            628
E            595
W            574
SSE          515
NW           478
VAR          473
WNW          453
SW           448
SSW          448
NNW          429
WSW          427
SE           424
ENE          419
NNE          363
ESE          362
NE           359
North         14
East           6
Variable       6
South          5
West           5
Name: count, dtype: int64

I can see that we have values which are the same but recorded differently as shorthand or longhand, for example, "S" and "South". I will create a map to convert the longhand to shorthand version.

In [28]:
# define mapping
direction_map = {
    "Variable": "VAR",
    "South": "S",
    "North": "N",
    "West": "W",
    "East": "E"
}

# apply mapping
df["Wind_Direction"] = df["Wind_Direction"].replace(direction_map)

# check unique values again
df["Wind_Direction"].unique()

array(['CALM', 'N', 'SSE', 'E', 'S', 'WNW', 'NNW', 'SE', 'ENE', 'NW',
       'NE', 'WSW', 'NNE', 'SW', 'W', 'ESE', 'VAR', 'SSW'], dtype=object)

Next, I will look at values for "Weather_Condition".

In [29]:
df["Weather_Condition"].value_counts()

Weather_Condition
Fair                       4718
Cloudy                     1598
Mostly Cloudy              1308
Partly Cloudy               808
Light Rain                  453
Light Snow                  199
Fog                         160
Rain                        119
Haze                         83
Fair / Windy                 65
Cloudy / Windy               45
Heavy Rain                   41
Mostly Cloudy / Windy        37
Snow                         32
Thunder in the Vicinity      32
Smoke                        29
Overcast                     28
Light Drizzle                27
Thunder                      24
Wintry Mix                   24
T-Storm                      18
Light Rain / Windy           17
Partly Cloudy / Windy        16
Light Rain with Thunder      14
Light Snow / Windy           13
Heavy T-Storm                11
Heavy Snow                    9
Shallow Fog                   6
Light Freezing Rain           5
T-Storm / Windy               5
Mist                  

I am going to create a new column, "Weather_Simplified" to reduce and simplify the number of different types of weather conditions. I will create a csv file that maps each "Weather_Condition" to "Weather_Simplified" and then merge "Weather_Simplified" as a new column in DataFrame.  

"Weather_Simplified" will be constructed such that all types of rain or snow or fog etc. are grouped together. Mixed weather conditions were consistently found to be "Condition"/ "Windy". In this case, all simplified to "Condition" with "Windy" dropped, unless the condition was "Fair", "Cloudy" or "Mostly Cloudy", then it simplified to "Windy". This was to record the 'worst' of the mixed conditions, which is a subjective point of view and should be reviewed with the client. 

In [16]:
# Get unique simplified weather conditions
unique_conditions = df['Weather_Condition'].dropna().unique()

# Convert to a DataFrame for better Excel paste
unique_df = pd.DataFrame(unique_conditions, columns=['Weather_Condition'])

# Copy to clipboard
unique_df.to_clipboard(index=False)  # No index column
print("Copied to clipboard! You can now paste into Excel.")

Copied to clipboard! You can now paste into Excel.


In [30]:
weather_map = pd.read_csv("Data/Raw/Supporting_files/Weather_Condition_Map.csv")
weather_map.head()

,Weather_Condition,Weather_Simplified
0,Fair,Fair
1,Mostly Cloudy,Cloudy
2,Cloudy,Cloudy
3,Partly Cloudy,Rain
4,Light Rain,Rain


In [31]:
df = df.merge(weather_map, on="Weather_Condition", how="left")

# check if any Weather_Condition values didn't get mapped
unmapped = df[df["Weather_Simplified"].isna()]["Weather_Condition"].unique()

if len(unmapped) > 0:
    print("Warning: The following Weather_Condition values were not mapped:")
    print(unmapped)
else:
    print("All Weather_Condition values successfully mapped.")

All Weather_Condition values successfully mapped.


In [32]:
df["Weather_Simplified"].value_counts()

Weather_Simplified
Fair      4719
Cloudy    2934
Rain      1485
Snow       263
Fog        174
Windy      163
Storm      107
Haze        87
Smoke       32
Wintry      24
Ice          7
Mist         5
Name: count, dtype: int64

Next, I will look at the values for "State".

In [33]:
df["State"].value_counts()

State
CA    1820
FL    1493
LA     739
TX     562
OR     477
NY     421
SC     375
PA     353
NC     349
AZ     316
VA     313
OK     273
TN     188
MN     184
IL     154
NJ     151
WA     150
GA     143
MT     141
MI     136
MD     135
AL     123
CO     107
OH     106
UT      89
CT      83
MO      64
MA      60
WV      50
IN      47
KS      46
NV      38
DE      36
DC      33
IA      30
NE      30
WI      28
AR      27
ID      26
WY      25
KY      24
MS      20
NM      16
RI      11
NH       3
ME       2
ND       2
VT       1
Name: count, dtype: int64

I am going to create a new column, "State_Other", where States that appear less than 5 times are grouped as "Other", otherwise the State appears as it is in "State". 

In [ ]:
counts = df["State"].value_counts()

df["State_Other"] = df["State"].apply(lambda x: x if counts[x] > 5 else "Other")

df["State_Other"].unique()

array(['LA', 'VA', 'FL', 'OK', 'PA', 'KY', 'OR', 'AZ', 'MT', 'NJ', 'NE',
       'NY', 'NC', 'CA', 'KS', 'WY', 'TN', 'SC', 'MN', 'GA', 'MS', 'CO',
       'ID', 'WA', 'DE', 'UT', 'TX', 'OH', 'MO', 'NM', 'MI', 'IA', 'IL',
       'DC', 'NV', 'IN', 'WV', 'AL', 'CT', 'WI', 'MD', 'AR', 'ME', 'MA',
       'RI', 'NH', 'VT', 'ND'], dtype=object)

Next, I will look at how many unique values there are for "City" and how many appear only once.

In [35]:
df["City"].nunique()

2433

In [36]:
# Count occurrences of each city
city_counts = df['City'].value_counts()

# Count how many cities appear exactly once
low_cities_count = (city_counts < 2).sum()

print(f"Number of cities that appear only once: {low_cities_count}")

Number of cities that appear only once: 1266


I believe there are too many cities that appear only once to either leave them in or group as "Other". Instead, I will use "City" to extract population counts and then ultimately drop this variable.

I have found both city and county population data which I have downloaded from the United States Census Bureau. I will first attempt to map "Population" using "City", and for those not found, using "County".

I will read in the csv file for city populations.

In [42]:
df_city_pop = pd.read_csv("Data/Raw//Supporting_files/City_Pop_Map.csv", encoding='latin1')
df_city_pop

,Area,Population
0,"Abbeville city, Alabama","2,349"
1,"Adamsville city, Alabama","4,393"
2,"Addison town, Alabama",661
3,"Akron town, Alabama",229
4,"Alabaster city, Alabama","33,342"
...,...,...
21408,NaN,NaN
21409,NaN,NaN
21410,NaN,NaN
21411,NaN,NaN


I will drop rows which have only "NaN".

In [43]:
df_city_pop = df_city_pop.dropna(how='all')
df_city_pop

,Area,Population
0,"Abbeville city, Alabama","2,349"
1,"Adamsville city, Alabama","4,393"
2,"Addison town, Alabama",661
3,"Akron town, Alabama",229
4,"Alabaster city, Alabama","33,342"
...,...,...
19474,"Wamsutter town, Wyoming",203
19475,"Wheatland town, Wyoming","3,586"
19476,"Worland city, Wyoming","4,784"
19477,"Wright town, Wyoming","1,645"


I can see that city/ towns are together with their states, separated by a comma under the heading "Area". I will create a new column "City" for city/ towns, and a new column "State" for the states. I will also strip the word "city" from the names because they aren't recorded as "... city" in the US_Accidents dataset. Lastly, I will drop the column "Area".

In [44]:
df_city_pop = df_city_pop.copy()

# Remove leading/trailing spaces 
df_city_pop['Area'] = df_city_pop['Area'].str.strip()

# Split at the last comma into city and state
df_city_pop[['City', 'State']] = df_city_pop['Area'].str.rsplit(',', n=1, expand=True)

# Strip extra spaces and remove "city" in the name
df_city_pop['City'] = df_city_pop['City'].str.replace(r'\b[Cc]ity\b', '', regex=True).str.strip()
df_city_pop['State'] = df_city_pop['State'].str.strip()

# Optional: drop 'Area'
df_city_pop = df_city_pop.drop(columns=['Area'])

df_city_pop.head()

,Population,City,State
0,"2,349",Abbeville,Alabama
1,"4,393",Adamsville,Alabama
2,661,Addison town,Alabama
3,229,Akron town,Alabama
4,"33,342",Alabaster,Alabama


Next, I will create a new column "State_Abbrev" and map state abbreviations which match those found in the US_Accidents dataset.

In [45]:
# Dictionary of US states
state_map = {
    'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA',
    'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'District of Columbia':'DC',
    'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL',
    'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY', 'Louisiana':'LA',
    'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN',
    'Mississippi':'MS', 'Missouri':'MO', 'Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV',
    'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY',
    'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK', 'Oregon':'OR',
    'Pennsylvania':'PA', 'Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD',
    'Tennessee':'TN', 'Texas':'TX', 'Utah':'UT', 'Vermont':'VT', 'Virginia':'VA',
    'Washington':'WA', 'West Virginia':'WV', 'Wisconsin':'WI', 'Wyoming':'WY'
}

# Apply mapping
df_city_pop['State_Abbrev'] = df_city_pop['State'].map(state_map)
df_city_pop.head()

,Population,City,State,State_Abbrev
0,"2,349",Abbeville,Alabama,AL
1,"4,393",Adamsville,Alabama,AL
2,661,Addison town,Alabama,AL
3,229,Akron town,Alabama,AL
4,"33,342",Alabaster,Alabama,AL


Then, I will drop the "State" column so I can rename "State_Abbrev" to "State". This will allow me to match on both "City" and "State" when I merge DataFrames.

In [46]:
# Drop the original full state name column
df_city_pop = df_city_pop.drop(columns=['State'])

# Rename the abbreviation column
df_city_pop = df_city_pop.rename(columns={'State_Abbrev': 'State'})
df_city_pop.head()

,Population,City,State
0,"2,349",Abbeville,AL
1,"4,393",Adamsville,AL
2,661,Addison town,AL
3,229,Akron town,AL
4,"33,342",Alabaster,AL


Next, I will create a "City_norm" and "State_norm" column in each DataFrame to increase the chances of finding a match.

In [48]:
def normalize_city(name):
    name = name.lower().strip()
    
    # Remove parenthetical content, e.g., " (city)"
    name = re.sub(r"\s*\(.*?\)\s*", "", name)
    
    # Remove common suffixes
    remove_terms = [" city"]
    for term in remove_terms:
        name = name.replace(term, "")
    
    # Remove extra spaces
    name = re.sub(r"\s+", " ", name).strip()
    
    return name

# Apply to both datasets
df['City_norm'] = df['City'].apply(normalize_city)
df['State_norm'] = df['State'].str.lower().str.strip()

df_city_pop['City_norm'] = df_city_pop['City'].apply(normalize_city)
df_city_pop['State_norm'] = df_city_pop['State'].str.lower().str.strip()

Before mergining, I will take a look at how many cities have been matched compared with how many are not matched.

In [49]:
# Make sets of cities (and states) in each dataset
acc_cities = set(zip(df['City_norm'], df['State_norm']))
pop_cities = set(zip(df_city_pop['City_norm'], df_city_pop['State_norm']))

# Count how many cities in df_balanced are in df_pop
matched = acc_cities & pop_cities
print(f"Cities in df_balanced found in df_pop: {len(matched)}")
print(f"Cities in df_balanced NOT found in df_pop: {len(acc_cities - pop_cities)}")

Cities in df_balanced found in df_pop: 1383
Cities in df_balanced NOT found in df_pop: 1353


Next, I will read in the csv file containing county populations.

In [51]:
df_county_pop = pd.read_csv("Data/Raw/Supporting_files/County_Pop_Map.csv", encoding='latin1')
df_county_pop.head()

,County,Population
0,".Autauga County, Alabama","58,800"
1,".Baldwin County, Alabama","231,767"
2,".Barbour County, Alabama","25,226"
3,".Bibb County, Alabama","22,284"
4,".Blount County, Alabama","59,130"


Once, again, I need to formate and split "County" into "County" and "State" columns.

In [52]:
df_county_pop = df_county_pop.copy()

# Step 1: Remove leading "."
df_county_pop["County"] = df_county_pop["County"].str.lstrip(".")

# Step 2: Split into "County" and "State"
df_county_pop[["County", "State"]] = df_county_pop["County"].str.replace(" County", "", regex=False).str.rsplit(",", n=1, expand=True)

# Step 3: Clean up whitespace
df_county_pop["County"] = df_county_pop["County"].str.strip()
df_county_pop["State"] = df_county_pop["State"].str.strip()

# Check result
df_county_pop.head()


,County,Population,State
0,Autauga,"58,800",Alabama
1,Baldwin,"231,767",Alabama
2,Barbour,"25,226",Alabama
3,Bibb,"22,284",Alabama
4,Blount,"59,130",Alabama


Create "County_Abbrev" column.

In [53]:
# Dictionary of US states
state_map = {
    'Alabama':'AL', 'Alaska':'AK', 'Arizona':'AZ', 'Arkansas':'AR', 'California':'CA',
    'Colorado':'CO', 'Connecticut':'CT', 'Delaware':'DE', 'District of Columbia':'DC',
    'Florida':'FL', 'Georgia':'GA', 'Hawaii':'HI', 'Idaho':'ID', 'Illinois':'IL',
    'Indiana':'IN', 'Iowa':'IA', 'Kansas':'KS', 'Kentucky':'KY', 'Louisiana':'LA',
    'Maine':'ME', 'Maryland':'MD', 'Massachusetts':'MA', 'Michigan':'MI', 'Minnesota':'MN',
    'Mississippi':'MS', 'Missouri':'MO', 'Montana':'MT', 'Nebraska':'NE', 'Nevada':'NV',
    'New Hampshire':'NH', 'New Jersey':'NJ', 'New Mexico':'NM', 'New York':'NY',
    'North Carolina':'NC', 'North Dakota':'ND', 'Ohio':'OH', 'Oklahoma':'OK', 'Oregon':'OR',
    'Pennsylvania':'PA', 'Rhode Island':'RI', 'South Carolina':'SC', 'South Dakota':'SD',
    'Tennessee':'TN', 'Texas':'TX', 'Utah':'UT', 'Vermont':'VT', 'Virginia':'VA',
    'Washington':'WA', 'West Virginia':'WV', 'Wisconsin':'WI', 'Wyoming':'WY'
}

# Apply mapping
df_county_pop['State_Abbrev'] = df_county_pop['State'].map(state_map)
df_county_pop.head()

,County,Population,State,State_Abbrev
0,Autauga,"58,800",Alabama,AL
1,Baldwin,"231,767",Alabama,AL
2,Barbour,"25,226",Alabama,AL
3,Bibb,"22,284",Alabama,AL
4,Blount,"59,130",Alabama,AL


Then drop "State" so I can rename "State_Abbrev" as "State".

In [54]:
# Drop the original full state name column
df_county_pop = df_county_pop.drop(columns=['State'])

# Rename the abbreviation column
df_county_pop = df_county_pop.rename(columns={'State_Abbrev': 'State'})
df_county_pop.head()

,County,Population,State
0,Autauga,"58,800",AL
1,Baldwin,"231,767",AL
2,Barbour,"25,226",AL
3,Bibb,"22,284",AL
4,Blount,"59,130",AL


Again, I will create "County_norm" and "State_norm" to match counties in both DataFrames.

In [55]:
def normalize_county(name):
    name = name.lower().strip()
    
    # Remove parenthetical content, e.g., " (city)"
    name = re.sub(r"\s*\(.*?\)\s*", "", name)
    
    # Replace common suffixes
    remove_terms = [
        " county", " parish", " borough", " census area", 
        " independent city", " municipality", " district",
        " planning region", " region"
    ]
    for term in remove_terms:
        name = name.replace(term, "")
    
    # Normalize "st." or "st" to "st"
    name = re.sub(r"\bst\.?", "st", name)
    
    # Remove extra spaces
    name = re.sub(r"\s+", " ", name).strip()
    
    return name

# Apply to dataset
df['County_norm'] = df['County'].apply(normalize_county)

df_county_pop['County_norm'] = df_county_pop['County'].apply(normalize_county)
df_county_pop['State_norm'] = df_county_pop['State'].str.lower().str.strip()

In [ ]:
# Make sets of counties (and states) in each dataset
acc_counties = set(zip(df['County_norm'], df['State_norm']))
pop_counties = set(zip(df_county_pop['County_norm'], df_county_pop['State_norm']))

# Count how many cities in df_balanced are in df_pop
matched = acc_counties & pop_counties
print(f"Counties in df found in df_county_pop: {len(matched)}")
print(f"Counties in df NOT found in df_county_pop: {len(acc_counties - pop_counties)}")

# show first 20 unmatched pairs to inspect
list(acc_counties - pop_counties)[:20]

Counties in df found in df_county_pop: 953
Counties in df NOT found in df_county_pop: 24


[('suffolk', 'va'),
 ('fredericksburg', 'va'),
 ('charlottesville', 'va'),
 ('winchester', 'va'),
 ('de kalb', 'tn'),
 ('macon-bibb', 'ga'),
 ('bristol', 'va'),
 ('harrisonburg', 'va'),
 ('fairfield', 'ct'),
 ('virginia beach', 'va'),
 ('chesapeake', 'va'),
 ('new london', 'ct'),
 ('alexandria', 'va'),
 ('norfolk', 'va'),
 ('queen annes', 'md'),
 ('colonial heights', 'va'),
 ('middlesex', 'ct'),
 ('prince georges', 'md'),
 ('hartford', 'ct'),
 ('newport news', 'va')]

Before merging the DataFrames, I am checking for duplicates that may have arisen due to the actions of normalisation.

In [57]:
# Check df_pop for duplicate city-state pairs
dupes_city_pop = df_city_pop[df_city_pop.duplicated(subset=['City_norm', 'State_norm'], keep=False)]
print(f"Duplicate city-state pairs in df_pop: {dupes_city_pop.value_counts().sum()}")
print(dupes_city_pop.sort_values(['City_norm', 'State_norm']))

# Check df_cpop for duplicate county-state pairs
dupes_county_pop = df_county_pop[df_county_pop.duplicated(subset=['County_norm', 'State_norm'], keep=False)]
print(f"Duplicate county-state pairs in df_cpop: {dupes_county_pop.value_counts().sum()}")
print(dupes_county_pop.sort_values(['County_norm', 'State_norm']))

Duplicate city-state pairs in df_pop: 49
      Population                   City State              City_norm  \
3269       4,714        Beecher village    IL        beecher village   
3270         429       Beecher  village    IL        beecher village   
14706        177    Centerville borough    PA    centerville borough   
14707      3,257    Centerville borough    PA    centerville borough   
16718      2,853            Clarksville    TX            clarksville   
16719        768            Clarksville    TX            clarksville   
6686       1,030                   Clay    KY                   clay   
6687       1,194                   Clay    KY                   clay   
14740        129       Coaldale borough    PA       coaldale borough   
14741      2,427       Coaldale borough    PA       coaldale borough   
18976        234          Genoa village    WI          genoa village   
18977      2,982         Genoa  village    WI          genoa village   
14994        253      J

I can see that there are 49 City-State pair duplicates but 0 Couty-State pair duplicates. I am going to delete the City-State duplicates to prevent errors when merging.

In [58]:
df_city_pop_clean = df_city_pop[~df_city_pop.index.isin(dupes_city_pop.index)].copy()

Now, I will merge on "City_norm" and "State_norm" to get "Population_city". I'm checking the shape after merging to ensure that duplications aren't occurring.

In [59]:
# Merge city population
df = df.merge(df_city_pop_clean[['City_norm','State_norm','Population']],
                                left_on=['City_norm','State_norm'],
                                right_on=['City_norm','State_norm'],
                                how='left')
df.rename(columns={'Population':'Population_city'}, inplace=True)

df.shape

(10000, 42)

Next, I'm merging on "County_norm" and "State_norm", and again checking the shape after merging.

In [60]:
# Merge county population
df = df.merge(df_county_pop[['County_norm','State_norm','Population']],
                                left_on=['County_norm','State_norm'],
                                right_on=['County_norm','State_norm'],
                                how='left')
df.rename(columns={'Population':'Population_county'}, inplace=True)

df.shape

(10000, 43)

Finally, I'm creating the "Population" column that I will keep, which takes "Population_city" if available, otherwise it fills with "Population_county".

In [61]:
# Fill final population using city first, then County
df['Population'] = df['Population_city'].fillna(df['Population_county'])

df.shape

(10000, 44)

We can see that in the end, there are 40 instances without a value for "Population", which I will drop.

In [63]:
print(df["Population_city"].isna().sum())
print(df["Population_county"].isna().sum())
print(df["Population"].isna().sum())

2597
125
40


In [64]:
df = df.dropna(subset=['Population'])
df.shape

(9960, 44)

Lastly, I will need to strip the comma from the values of "Population" and change the data type from object to integer.

In [65]:
df["Population"].dtype

dtype('O')

In [66]:
df["Population"] = df["Population"].str.replace(",", "", regex=False).astype(int)
df["Population"].dtype

dtype('int32')

Lastly, I will look at the boolean variables and see if any should be dropped.

In [6]:
for col in df.select_dtypes(include="boolean").columns:
    print(f"{col}: {df[col].nunique()}")
    print(f"{df[col].unique()}")

Amenity: 2
[False  True]
Bump: 2
[False  True]
Crossing: 2
[False  True]
Give_Way: 2
[False  True]
Junction: 2
[False  True]
No_Exit: 2
[False  True]
Railway: 2
[False  True]
Roundabout: 1
[False]
Station: 2
[False  True]
Stop: 2
[False  True]
Traffic_Calming: 2
[False  True]
Traffic_Signal: 2
[False  True]
Turning_Loop: 1
[False]


---

## Conclusion and Next Steps

* In cases where you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.